In [1]:
Output = c('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.1. Data Cleaning, Imputation & Normalization/Output')
cur_date = "070423"

library(readxl)
library(tidyverse)
library(openxlsx)

# reading in files
proteomics_5_df = data.frame(read_excel("Input/Proteomics_Data_5ug_040423.xlsx", sheet = 2))
proteomics_25_df = data.frame(read_excel("Input/Proteomics_Data_25ug_040423.xlsx", sheet = 2))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
head(proteomics_25_df)

,Subject_No,Subject_ID,Protein_Accession,Gene_Name,Description,Condensate,Burn_Condition,Concentration,Intensity,Intensity_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,1488600000,30.47131
2,2,M_2,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3806600000,31.82586
3,3,M_3,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3900000000,31.86083
4,4,F_4,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5373200000,32.32313
5,5,F_5,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,2595200000,31.27320
6,6,M_6,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5780700000,32.42860


# Peptide Normalization
Normalize to total amount of peptide per subject: take the intensity value for each protein for each subject, then calculate the median value, and calculate the ratio.

In [3]:
# creating 1 df
proteomics_df = rbind(proteomics_25_df, proteomics_5_df)

normalized_df = proteomics_df %>%
    group_by(Concentration, Condensate, Burn_Condition, Subject_ID) %>%
    #removing missing data (to be added back in later)
    drop_na() %>%
    mutate(Summed_Intensity = sum(Intensity)) %>%
    #calculating median across all subjects but still within each set
    ungroup() %>%
    mutate(Median_of_Sum = median(Summed_Intensity), Norm_Factor = Summed_Intensity/ Median_of_Sum, 
           Norm_Intensity = Intensity/Norm_Factor, Norm_Intensity_pslog2 = log2(Norm_Intensity + 1)) %>%
    select(-c("Summed_Intensity", "Median_of_Sum", "Norm_Factor", "Intensity", "Intensity_pslog2"))

head(normalized_df)

Subject_No,Subject_ID,Protein_Accession,Gene_Name,Description,Condensate,Burn_Condition,Concentration,Norm_Intensity,Norm_Intensity_pslog2
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,F_1,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,1657403040,30.62628
2,M_2,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,1104030254,30.04013
3,M_3,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3833400337,31.83598
4,F_4,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3751240231,31.80472
5,F_5,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3011320293,31.48775
6,M_6,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,6331729010,32.55995


In [4]:
# splitting into 2 dfs again to decrease the file size
split_proteomics_df = normalized_df %>%
    group_by(Concentration) %>%
    group_split()

norm_proteomics_5_df = split_proteomics_df[[1]]
norm_proteomics_25_df = split_proteomics_df[[2]]

In [5]:
# exporting
write.xlsx(norm_proteomics_25_df, paste0(Output,"/", "Normalized_Proteomics_Data_25ug_", cur_date, ".xlsx"), 
           rowNames = FALSE)
write.xlsx(norm_proteomics_5_df, paste0(Output,"/", "Normalized_Proteomics_Data_5ug_", cur_date, ".xlsx"), 
           rowNames = FALSE)